In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
import numpy as np

In [3]:
import wandb
from wandb.integration.keras import WandbMetricsLogger, WandbModelCheckpoint
wandb.require("core")
wandb.login()

wandb: WARNING `wandb.require('core')` is a no-op as it is now the default behavior.
wandb: Currently logged in as: emmdaz (emmdaz-zzz) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [42]:
wandb.init(
    project="Experiments Series 1.01",
    config={
        "Layer1": 128,
        "Activation_1": "sigmoid",
        "Dropout1": "No dropout",
        "Layer2": 256,
        "Activation_2": "sigmoid",
        "Dropout2": "No dropout",
        "Layer3": 10,
        "Activation_3": "softmax",
        "Dropout3": "No dropout",
        "Optimizer": "sgd",
        "Metric": "accuracy",
        "Epoch": 30,
        "Batch_size": 20,
        "Eta": 1e-3,
        "Loss": "binary_crossentropy"
    }
)

In [43]:
config = wandb.config

In [44]:
ds=mnist.load_data()

In [45]:
(x_train,y_train),(x_test,y_test)=ds
print(x_train.shape,x_test.shape)

(60000, 28, 28) (10000, 28, 28)


In [46]:
# Se hace al aplanado de las imágenes
x_train=x_train.reshape(60000,784)
x_test=x_test.reshape(10000,784)

# Se modifica el tipo de datos a tipo flotante (para evitar conflictos)
x_train = x_train.astype("float32")
x_test = x_test.astype("float32")

# Se normalizan los datos acorde a la intensidad de los píxeles
x_train /= 255.
x_test /= 255.

In [47]:
# Se realiza el one-hot enconding:

# Se definen el número de clases que hay en la base de datos
classes = config.Layer3

# Se realiza la conversión:
y_train = keras.utils.to_categorical(y_train,classes)
y_test = keras.utils.to_categorical(y_test,classes)

In [48]:
# Se configuran los hiperparámetros del modelo:

# Valor del Learning Rate
eta = config.Eta
eta = float(eta) # Tiene que ser un dato de tipo flotante, pues de lo contrario
                 # tiene conflicto

# Número de épocas
epochs = config.Epoch

# Tamaño del mini-batch
mini_batch= config.Batch_size

In [49]:
# Se realiza la creación del modelo a como se había definido usando Numpy:
model = Sequential([
    Dense(config.Layer1, activation = config.Activation_1, input_shape = (784,)),
    Dense(config.Layer2, activation = config.Activation_2),
    Dense(classes, activation = config.Activation_3)
])
model.summary()
# El modelo es secuencial, se van uniendo neurona por neurona de forma consecutiva

/home/emmdzzz/anaconda3/lib/python3.13/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_10 (Dense)                │ (None, 128)            │       100,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 256)            │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 10)             │         2,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 136,074 (531.54 KB)

 Trainable params: 136,074 (531.54 KB)

 Non-trainable params: 0 (0.00 B)

In [50]:
model.compile(loss=config.Loss, optimizer = config.Optimizer,
              metrics=[config.Metric])

history = model.fit(x_train, y_train,
                    batch_size = mini_batch,
                    epochs = epochs,
                    verbose = 1,
                    validation_data = (x_test, y_test),
                    callbacks=[WandbMetricsLogger(log_freq=5),
                               WandbModelCheckpoint("models.keras")]
                    )

Epoch 1/30
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.1114 - loss: 0.3436 - val_accuracy: 0.1135 - val_loss: 0.3230
Epoch 2/30
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 14s 4ms/step - accuracy: 0.1532 - loss: 0.3224 - val_accuracy: 0.1178 - val_loss: 0.3203
Epoch 3/30
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.1934 - loss: 0.3197 - val_accuracy: 0.2228 - val_loss: 0.3173
Epoch 4/30
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - accuracy: 0.3465 - loss: 0.3165 - val_accuracy: 0.4736 - val_loss: 0.3136
Epoch 5/30
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.4467 - loss: 0.3127 - val_accuracy: 0.5625 - val_loss: 0.3089
Epoch 6/30
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.5500 - loss: 0.3078 - val_accuracy: 0.5586 - val_loss: 0.3027
Epoch 7/30
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.5857 - loss: 0.3013 - val_accuracy: 0.6095 - val_loss: 0.2945
Epoch 8/30
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - accuracy: 0.5979 - loss: 0.2925 

In [53]:
wandb.finish()

In [52]:
mod = model.predict(x_test)

print("Resultado predicho: ")
print(np.where(mod[6] == np.max(mod[6])))
print("Resultado correcto: ")
print(np.where(y_test[6] == np.max(y_test[6])))


313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step   
Resultado predicho: 
(array([9]),)
Resultado correcto: 
(array([4]),)
